In [1]:
import numpy as np, matplotlib.pyplot as plt, pandas as pd
# For finding and loading multiple files
import os
# Use this to see how much memory the dataframes use
from sys import getsizeof
# To view keyspace.yaml
import yaml
from pathlib import Path

import pseudopeople as psp

from vivarium_research_prl.utils import sizemb, MappingViaAttributes, build_full_address
from vivarium_research_prl import alpha

!date
!whoami
!uname -a
!python --version
!pwd

Fri 18 Aug 2023 02:45:27 PM PDT
ndbs
Linux long-slurm-sarchive-p0048 5.4.0-135-generic #152-Ubuntu SMP Wed Nov 23 20:19:22 UTC 2022 x86_64 x86_64 x86_64 GNU/Linux
Python 3.10.12
/mnt/share/code/ndbs/vivarium_research_prl/pseudopeople_testing


In [26]:
%load_ext autoreload
%autoreload 2

# Things to test

1. Do all the `generate_x` functions work when passed either a string or a Path object?
2. Do all the datasets have the right columns?
3. Do the generate functions work when `NO_NOISE` is passed? Do they run faster?
4. 

# Find data

OLD USA:

`/mnt/team/simulation_science/priv/engineering/vivarium_census_prl_synth_pop
/results/release_02_yellow/full_data
/united_states_of_america/2023_07_28_08_33_09
/final_results/2023_08_02_11_38_10/pseudopeople_input_data_usa`

NEW USA:

`/mnt/team/simulation_science/priv/engineering/vivarium_census_prl_synth_pop
/results/release_02_yellow/full_data
/united_states_of_america/2023_07_28_08_33_09
/final_results/2023_08_16_09_58_54/pseudopeople_input_data_usa`

NEW RI:

`/mnt/team/simulation_science/priv/engineering/vivarium_census_prl_synth_pop
/results/release_02_yellow/full_data
/united_states_of_america/2023_07_28_08_33_09
/final_results/2023_08_16_09_58_54/states/pseudopeople_input_data_rhode_island`

In [4]:
project_dir = Path('/mnt/team/simulation_science/priv/engineering/vivarium_census_prl_synth_pop')
full_data_dir = project_dir / 'results/release_02_yellow/full_data'
usa_model_dir = full_data_dir / 'united_states_of_america/2023_07_28_08_33_09'
usa_data_dir = usa_model_dir / 'final_results/2023_08_16_09_58_54/pseudopeople_input_data_usa'
ri_data_dir = usa_model_dir / 'final_results/2023_08_16_09_58_54/states/pseudopeople_input_data_rhode_island'

!ls -halt $ri_data_dir

total 228K
drwxrwsr-x 9 albrja IHME-Simulationscience 3.5K Aug 17 19:16 .
drwxrwsr-x 2 albrja IHME-Simulationscience    0 Aug 17 19:16 taxes_dependents
drwxrwsr-x 2 albrja IHME-Simulationscience 167K Aug 17 19:16 taxes_1040
drwxrwsr-x 2 albrja IHME-Simulationscience 167K Aug 17 16:39 taxes_w2_and_1099
drwxrwsr-x 2 albrja IHME-Simulationscience 167K Aug 17 14:20 women_infants_and_children
drwxrwsr-x 2 albrja IHME-Simulationscience 167K Aug 17 14:13 current_population_survey
drwxrwsr-x 2 albrja IHME-Simulationscience 167K Aug 17 14:08 american_community_survey
drwxrwsr-x 2 albrja IHME-Simulationscience 167K Aug 17 14:06 decennial_census
drwxrwsr-x 3 albrja IHME-Simulationscience  512 Aug 17 13:13 ..


In [7]:
!ls -halt $usa_model_dir

total 1.2M
drwxrwsr-x  5 sbachmei IHME-Simulationscience 2.5K Aug 16 11:45 ..
drwxrwsr-x  7 sbachmei IHME-Simulationscience 3.5K Aug 16 11:45 final_results
drwxrwsr-x  6 sbachmei IHME-Simulationscience 5.0K Jul 31 08:59 .
-rw-rw-r--  1 sbachmei IHME-Simulationscience 1.2M Jul 29 06:01 output.hdf
drwxrwsr-x 10 sbachmei IHME-Simulationscience 4.0K Jul 28 17:09 raw_results
drwxrwsr-x  2 sbachmei IHME-Simulationscience  512 Jul 28 08:33 __pycache__
-rw-rw-r--  1 sbachmei IHME-Simulationscience  110 Jul 28 08:33 settings.py
-rw-rw-r--  1 sbachmei IHME-Simulationscience 1.9K Jul 28 08:33 model_specification.yaml
-rw-rw-r--  1 sbachmei IHME-Simulationscience   50 Jul 28 08:33 branches.yaml
-rw-rw-r--  1 sbachmei IHME-Simulationscience 2.4K Jul 28 08:33 keyspace.yaml
-rw-rw-r--  1 sbachmei IHME-Simulationscience 4.2K Jul 28 08:33 requirements.txt
drwxrwsr-x  3 sbachmei IHME-Simulationscience  512 Jul 28 08:33 logs


# Try generating some RI data

In [8]:
%%time
acs = psp.generate_american_community_survey(ri_data_dir)
acs

Applying noise: 100%|█████████████████████████████████████████████████████████████| 14/14 [00:00<00:00, 29.08type/s]


CPU times: user 3min 2s, sys: 3.86 s, total: 3min 6s
Wall time: 3min 8s


,simulant_id,household_id,survey_date,first_name,middle_initial,last_name,age,date_of_birth,street_number,street_name,unit_number,city,state,zipcode,relationship_to_reference_person,sex,race_ethnicity
0,1061_245665,1061_98878,2020-03-24,Lorin,R,Zieba,61,11/03/1958,8114,w 127th pl,NaN,bristol,RI,02813,Reference person,Male,White
1,1061_245666,1061_98878,2020-03-24,Margaret,S,Zieba,60,08/01/1959,8114,w 127th pl,NaN,bristol,RI,02813,Opp-sex spouse,Female,White
2,1061_754485,1061_303835,2020-05-19,Jeremy,I,Stoll,46,01/31/1974,28,hampton road,NaN,newport,RI,02895,Reference person,Male,White
3,1061_754488,1061_303835,2020-05-19,Kaylee,H,Stoll,10,10/26/2009,28,hampton road,NaN,newport,RI,02895,Biological child,Female,Asian
4,107_473988,107_190594,2020-03-24,Joseph,J,Gilder,41,02/05/1979,1450,siberian elm ln,NaN,lincoln,RI,02895,Opp-sex spouse,Male,Multiracial or Other
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1128,9971_30032,9971_12083,2020-03-24,Shawna,J,Blackmon,38,02/05/1982,3654,farris rd,NaN,johnston,RI,02865,Other relative,Female,White
1129,9971_944282,9971_379364,2020-03-24,Anna,V,Muller,38,07/15/1981,132,apache ave,NaN,east greenwich,RI,02863,Reference person,Female,White
1130,9971_944283,9971_379364,2020-03-24,Timothy,L,Muller,36,01/21/1984,132,apache ave,NaN,east greenwich,RI,02863,Opp-sex spouse,Male,White
1131,9971_186996,9971_75321,2020-08-11,Josiah,B,Francis,5,06/30/2015,NaN,sunset street,NaN,cumberland,RI,02906,Reference person,Male,Asian


In [9]:
acs.memory_usage(deep=True)

Index                                 128
simulant_id                         76833
household_id                        76595
survey_date                          9064
first_name                          71364
middle_initial                      65714
last_name                           71986
age                                 40720
date_of_birth                       75911
street_number                       67196
street_name                         78420
unit_number                         38297
city                                75970
state                                6246
zipcode                             70246
relationship_to_reference_person     3036
sex                                  1365
race_ethnicity                       1882
dtype: int64

In [13]:
acs.memory_usage(deep=True).sum()

830973

# Try generating RI social security data

In [16]:
%%time
ssa = psp.generate_social_security(ri_data_dir)
ssa

DataSourceError: No datasets found at directory /mnt/team/simulation_science/priv/engineering/vivarium_census_prl_synth_pop/results/release_02_yellow/full_data/united_states_of_america/2023_07_28_08_33_09/final_results/2023_08_16_09_58_54/states/pseudopeople_input_data_rhode_island. Please provide the path to the unmodified root data directory.

In [25]:
try:
    ssa = psp.generate_social_security(ri_data_dir)
except BaseException as e:
    print('Goat')
    ssa = e
    print('Yak!')
ssa

Goat
Yak!


DataSourceError(message='No datasets found at directory /mnt/team/simulation_science/priv/engineering/vivarium_census_prl_synth_pop/results/release_02_yellow/full_data/united_states_of_america/2023_07_28_08_33_09/final_results/2023_08_16_09_58_54/states/pseudopeople_input_data_rhode_island. Please provide the path to the unmodified root data directory.')

In [27]:
ssa

DataSourceError(message='No datasets found at directory /mnt/team/simulation_science/priv/engineering/vivarium_census_prl_synth_pop/results/release_02_yellow/full_data/united_states_of_america/2023_07_28_08_33_09/final_results/2023_08_16_09_58_54/states/pseudopeople_input_data_rhode_island. Please provide the path to the unmodified root data directory.')

# Try generating RI 1040 data

In [30]:
ri_data_dir

PosixPath('/mnt/team/simulation_science/priv/engineering/vivarium_census_prl_synth_pop/results/release_02_yellow/full_data/united_states_of_america/2023_07_28_08_33_09/final_results/2023_08_16_09_58_54/states/pseudopeople_input_data_rhode_island')

In [ ]:
%%time
t1040 = psp.generate_taxes_1040(ri_data_dir)
t1040

Noising data:  34%|███████████████████▉                                       | 113/334 [1:20:13<2:36:45, 42.56s/it]

# Try writing a version of `generate_datasets` that will catch exceptions on individual datasets so the whole thing doesn't fail

In [ ]:
from psp.exceptions import ConfigurationError, DataSourceError

In [ ]:
def generate_datasets1(*args, **kwargs) -> MappingViaAttributes:
    """Generate all pseudopeople datasets and return them in a MappingViaAttributes
    mapping, which is a light wrapper for a dictionary enabling easy tab completion
    of dict keys. The keys will be the strings that appear after 'generate_' in the
    function names, and the values will be the datasets. `args` and `kwargs` are
    passed to each of the dataset generation functions.
    """
    generation_fns = (getattr(psp, name) for name in dir(psp) if 'generate' in name)
    # data = {f.__name__.replace('generate_', ''): f(*args, **kwargs) for f in generation_fns}
    data = {}
    for f in generation_fns:
        dataset_name = f.__name__.replace('generate_', '')
        try:
            data[dataset_name] = f(*args, **kwargs)
        except (ConfigurationError, DataSourceError, Exception) as e:
            data[dataset_name] = e
    return MappingViaAttributes(data)

def generate_datasets2(*args, **kwargs) -> MappingViaAttributes:
    """Generate all pseudopeople datasets and return them in a MappingViaAttributes
    mapping, which is a light wrapper for a dictionary enabling easy tab completion
    of dict keys. The keys will be the strings that appear after 'generate_' in the
    function names, and the values will be the datasets. `args` and `kwargs` are
    passed to each of the dataset generation functions.
    """
    def value_or_error(f):
        try:
            return f(*args, **kwargs)
        except (ConfigurationError, DataSourceError, Exception) as e:
            return e

    generation_fns = (getattr(psp, name) for name in dir(psp) if 'generate' in name)
    data = {f.__name__.replace('generate_', ''): value_or_error(f) for f in generation_fns}
    return MappingViaAttributes(data)